In [1]:
import tensorflow as tf
import tflearn
import numpy as np
import gensim
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
from gensim import corpora
from pprint import pprint
import pickle
WORD_SIZE = 5
MAX_FILTER_SIZE = 6
START_WORD_VEC = np.asarray([1,0], dtype=np.float32).reshape([1,2])
END_WORD_VEC = np.asarray([0,1], dtype=np.float32).reshape([1,2])
END_WORD_ID = 1
START_WORD_ID = 0

EMBEDDING_MATRIX, ID2WORD, WORD2ID = pickle.load(open("embedding_matrix_2D",'rb'))


In [2]:
def id2onehot(word_id):
    vec = np.zeros([WORD_SIZE], dtype=np.float32)
    vec[word_id] = 1.0
    return vec
def onehot2id(vec):
    index = vec.argsort()[-1]
    return index

def embedding_look_up(word_id):
    word = WORD_DIC.get(word_id)
    return word_model[word]

In [3]:
class LSTM:
    def __init__(self, vocabulary_size, num_nodes, 
                 batch_size, name=""):
        '''
        the LSTM process
        '''
        self.num_nodes = num_nodes
        self.reuse = False
        self.vocabulary_size = vocabulary_size
        self.num_nodes = num_nodes
        self.batch_size = batch_size
        self.name = name
    
    def model(self, input_sentences, vocabulary_size, num_nodes,
             batch_size):
        #to make sure nothing goes wrong...
        assert input_sentences[0].shape[0] == batch_size
        
        with tf.variable_scope("{}_LSTM_var".format(self.name) ,reuse=self.reuse):
            with tf.variable_scope("input_gate"):
                self.ix = tf.get_variable("ix", [vocabulary_size, num_nodes], 
                                      tf.float32, tf.truncated_normal_initializer(stddev=0.1))
                self.im = tf.get_variable("im", [num_nodes, num_nodes], 
                                      tf.float32, tf.truncated_normal_initializer(stddev=0.1))
                self.ib = tf.get_variable("ib", [1, num_nodes], 
                                      tf.float32, tf.constant_initializer(0.1))
            with tf.variable_scope("forget_gate"):
                self.fx = tf.get_variable("fx", [vocabulary_size, num_nodes], tf.float32, 
                                     tf.truncated_normal_initializer(stddev=0.1))
                self.fm = tf.get_variable("fm", [num_nodes, num_nodes], 
                                      tf.float32, tf.truncated_normal_initializer(stddev=0.1))
                self.fb = tf.get_variable("fb", [1, num_nodes], 
                                     tf.float32, tf.constant_initializer(0.1))
            
            with tf.variable_scope("output_gate"):
                self.ox = tf.get_variable("ox", [vocabulary_size, num_nodes], tf.float32, 
                                     tf.truncated_normal_initializer(stddev=0.1))
                self.om = tf.get_variable("om", [num_nodes, num_nodes], 
                                      tf.float32, tf.truncated_normal_initializer(stddev=0.1))
                self.ob = tf.get_variable("ob", [1, num_nodes], 
                                     tf.float32, tf.constant_initializer(0.1))
            
            with tf.variable_scope("memory_cell"):
                self.cx = tf.get_variable("cx", [vocabulary_size, num_nodes], tf.float32, 
                                     tf.truncated_normal_initializer(stddev=0.1))
                self.cm = tf.get_variable("cm", [num_nodes, num_nodes], 
                                      tf.float32, tf.truncated_normal_initializer(stddev=0.1))
                self.cb = tf.get_variable("cb", [1, num_nodes], 
                                     tf.float32, tf.constant_initializer(0.1))
            
            self.saved_output = tf.get_variable("saved_output", [batch_size, num_nodes], 
                                                tf.float32, tf.constant_initializer(0), 
                                                trainable=False)
            self.saved_state = tf.get_variable("saved_state", [batch_size, num_nodes], 
                                                       tf.float32, tf.constant_initializer(0), 
                                                       trainable=False)
  
        self.train_data = input_sentences
    
        #Since now input_sentence is a list of word vectors, 
        #we do not need following 3 lines anymore.
        
        self.train_inputs = self.train_data
        
        self.outputs = list()
        self.output = self.saved_output
        self.state = self.saved_state
        
        self.state_list = list()
        self.state_list.append(self.saved_state)
        
        for i in self.train_inputs:
            self.output, self.state = self.lstm_cell(i, self.output, self.state)
            self.outputs.append(self.output)
            self.state_list.append(self.state)
            
        self.saved_state = self.saved_state.assign(self.state) 
        self.saved_output = self.saved_output.assign(self.output)
        
        self.variables = tf.get_collection(tf.GraphKeys.VARIABLES)
        
        self.reuse = True
        
        return self.outputs
    def lstm_cell(self, inputs, hidden_layer, state): 
        
        with tf.variable_scope("{}_LSTM_cell".format(self.name), reuse=self.reuse):
            input_gate = tf.sigmoid(tf.matmul(inputs, self.ix) + tf.matmul(hidden_layer, self.im) + self.ib,
                                    name="input_gate")
            forget_gate = tf.sigmoid(tf.matmul(inputs, self.fx) + tf.matmul(hidden_layer, self.fm) + self.fb,
                                    name="forget_gate")
            update_gate = tf.tanh(tf.matmul(inputs, self.cx) + tf.matmul(hidden_layer, self.cm) + self.cb,
                            name="update_gate")
            state = tf.add(tf.mul(forget_gate, state), tf.mul(input_gate, update_gate),
                           name="state")
            output_gate = tf.sigmoid(tf.matmul(inputs, self.ox) + tf.matmul(hidden_layer, self.om) + self.ob,
                                     name="output_gate") 
            output = tf.mul(output_gate, tf.tanh(state), name="output")
        
        #Note that, the "output" is equal to the hidden state
        return output, state   
    
    '''def lstm_process(self, inputs, hidden_layer, state):
        for i in train_inputs:
            self.output, self.state = self.lstm_cell(i, self.output, self.state)
            self.outputs.append(output)

        return outputs'''
    
    '''with tf.control_dependencies([self.saved_output.assign(output),
                                     self.saved_state.assign(state)]):
            
            logits = tf.matmul(tf.concat(0, outputs), self.w) + self.b
            loss = tf.reduce_mean(
                tf.nn.softmax_cross_entropy_with_logits(tf.concat(0, logits), 
                                                        tf.concat(0, train_labels), name="loss"))
            
            global_step = tf.Variable(0)
            learning_rate = tf.train.exponential_decay(
                10.0, global_step, 5000, 0.1, staircase=True)
            optimizer = tf.train.RMSPropOptimizer(learning_rate)
            gradient, v = zip(*optimizer.compute_gradients(loss))
            
            
            tf.add_to_collection("loss", loss)
            tf.add_to_collection("logits", logits)
            tf.add_to_collection("train_labels", train_labels)
            
        '''
        #It seems we do not need those lines above
        
    def __call__(self, input_sentences):
        return self.model(input_sentences, self.vocabulary_size, self.num_nodes,
                  self.batch_size)
        

In [4]:
L = LSTM(vocabulary_size=2, batch_size=1, num_nodes=4, name="TEST4")
word_model = gensim.models.Word2Vec.load("models/word2vec_2D.model")

test_word1 = np.reshape(word_model["china"], (1,2))
test_word2 = np.reshape(word_model["uk"], (1, 2))
test_word3 = np.reshape(word_model["america"], (1, 2))

test_sentence = [test_word1, test_word2, test_word3]

2016-10-15 16:31:57,330 : INFO : loading Word2Vec object from models/word2vec_2D.model
2016-10-15 16:31:57,564 : INFO : setting ignored attribute syn0norm to None
2016-10-15 16:31:57,567 : INFO : setting ignored attribute cum_table to None


In [5]:
with tf.Session() as sess:
    o = L(test_sentence)
    sess.run(tf.initialize_all_variables())
    o = sess.run(o)

In [6]:
o = L(test_sentence)

In [7]:
class gen_LSTM():
    def __init__(self, bi_hidden, batch_size, vocabulary_size, num_nodes, max_len=32, name=""):
        self.bi_hidden = bi_hidden
        self.name = name
        self.reuse = False
        self.max_len = max_len
        self.batch_size = batch_size
        self.vocabulary_size = vocabulary_size
        self.num_nodes = num_nodes
        
    def model(self, bi_hidden, batch_size, vocabulary_size, num_nodes, max_len):
        
        #assert bi_hidden[0].get_shape()[0] == batch_size
        #assert bi_hidden[0].get_shape()[1] == bi_hidden
        
        with tf.variable_scope("{}_gen_LSTM".format(self.name),
                              reuse=self.reuse):
            with tf.variable_scope("input_gate"):
                self.ix = tf.get_variable("ix", [vocabulary_size, num_nodes], 
                                      tf.float32, tf.truncated_normal_initializer(stddev=0.1))
                self.im = tf.get_variable("im", [num_nodes, num_nodes], 
                                      tf.float32, tf.truncated_normal_initializer(stddev=0.1))
                self.ib = tf.get_variable("ib", [1, num_nodes], 
                                      tf.float32, tf.constant_initializer(0.1))
                self.ic = tf.get_variable("ic", [num_nodes*2, num_nodes],
                                         tf.float32, tf.truncated_normal_initializer(stddev=0.1))
            with tf.variable_scope("forget_gate"):
                self.fx = tf.get_variable("fx", [vocabulary_size, num_nodes], tf.float32, 
                                     tf.truncated_normal_initializer(stddev=0.1))
                self.fm = tf.get_variable("fm", [num_nodes, num_nodes], 
                                      tf.float32, tf.truncated_normal_initializer(stddev=0.1))
                self.fb = tf.get_variable("fb", [1, num_nodes], 
                                     tf.float32, tf.constant_initializer(0.1))
                self.fc = tf.get_variable("fc", [num_nodes*2, num_nodes],
                                         tf.float32, tf.truncated_normal_initializer(stddev=0.1))
            
            with tf.variable_scope("output_gate"):
                self.ox = tf.get_variable("ox", [vocabulary_size, num_nodes], tf.float32, 
                                     tf.truncated_normal_initializer(stddev=0.1))
                self.om = tf.get_variable("om", [num_nodes, num_nodes], 
                                      tf.float32, tf.truncated_normal_initializer(stddev=0.1))
                self.ob = tf.get_variable("ob", [1, num_nodes], 
                                     tf.float32, tf.constant_initializer(0.1))
                self.oc = tf.get_variable("oc", [num_nodes*2, num_nodes],
                                         tf.float32, tf.truncated_normal_initializer(stddev=0.1))
            
            with tf.variable_scope("memory_cell"):
                self.cx = tf.get_variable("cx", [vocabulary_size, num_nodes], tf.float32, 
                                     tf.truncated_normal_initializer(stddev=0.1))
                self.cm = tf.get_variable("cm", [num_nodes, num_nodes], 
                                      tf.float32, tf.truncated_normal_initializer(stddev=0.1))
                self.cb = tf.get_variable("cb", [1, num_nodes], 
                                     tf.float32, tf.constant_initializer(0.1))
                self.cc = tf.get_variable("cc", [num_nodes*2, num_nodes],
                                         tf.float32, tf.truncated_normal_initializer(stddev=0.1))
                
            with tf.variable_scope("output_layer"):
                self.w = tf.get_variable("w", [num_nodes, WORD_SIZE], tf.float32,
                                        tf.truncated_normal_initializer(stddev=0.1))
                self.b = tf.get_variable("b", [WORD_SIZE], tf.float32,
                                        tf.constant_initializer(0))
                
            self.saved_output = tf.get_variable("saved_output", [batch_size, num_nodes], 
                                                tf.float32, tf.constant_initializer(0), 
                                                trainable=False)
            self.saved_state = tf.get_variable("saved_state", [batch_size, num_nodes], 
                                                       tf.float32, tf.constant_initializer(0), 
                                                       trainable=False)
        
            self.c = self.attention(self.bi_hidden, self.saved_state)
        
        self.variables = tf.get_collection(tf.GraphKeys.VARIABLES)
        
        self.init_state = self.saved_state
        self.state = self.saved_state
        self.output = self.saved_output
        self.states = list()
        self.states.append(self.saved_state)
        self.outputs = list()
        self.predicted_sentence = list()
        self.logits = list()
        self.reuse = True
        
        #generate words
        for i in range(self.max_len):
            if(i == 0):
                #input_word = id2onehot(START_WORD_ID).reshape([batch_size, WORD_SIZE])
                input_word = np.asarray([1.,0.]).reshape([batch_size, self.vocabulary_size])
                input_word = tf.cast(input_word, dtype=tf.float32)
                print("input word at step {}:{}".format(i, input_word))
            else:
                input_word = tf.nn.embedding_lookup(EMBEDDING_MATRIX, self.predicted_sentence[-1])
                input_word = tf.reshape(input_word, [batch_size, self.vocabulary_size])
                input_word = tf.cast(input_word, dtype=tf.float32)
                print("at step {} predicted word embedding:{}".format(i, input_word))
                
            self.output, self.state = self.gen_lstm_cell(input_word, self.output, self.state, self.c)
            self.outputs.append(self.output)
            self.states.append(self.state)
            logits = tf.matmul(self.output, self.w) + self.b
            self.logits.append(logits)
            prediction = tf.argmax(tf.nn.softmax(logits), 1)
            self.predicted_sentence.append(prediction[0])
            print("predicted word:{}".format(prediction))
            if tf.cast(tf.equal(prediction, END_WORD_ID), tf.int32) == 1:
                print("detected the END_WORD and break loop")
                break
            if i == max_len - 1:
                print("check point for max_len - 1")
                self.predicted_sentence.append(END_WORD_ID)
            
        
        self.saved_state = self.saved_state.assign(self.state)
        self.saved_output = self.saved_output.assign(self.output)
        
        #self.logits = tf.matmul(tf.concat(0, self.outputs), self.w) + self.b
        self.reuse = True
        return self.logits, self.predicted_sentence, self.states, self.saved_state
    
    def gen_lstm_cell(self, inputs, hidden_layer, state, c): 
        #single step LSTM with context vector
        with tf.variable_scope("{}_LSTM_cell".format(self.name), reuse=self.reuse):
            input_gate = tf.sigmoid(tf.matmul(inputs, self.ix) + tf.matmul(hidden_layer, self.im)\
                                    + tf.matmul(c, self.ic) + self.ib,
                                    name="input_gate")
            forget_gate = tf.sigmoid(tf.matmul(inputs, self.fx) + tf.matmul(hidden_layer, self.fm)\
                                    + tf.matmul(c, self.fc) + self.fb,
                                    name="forget_gate")
            update_gate = tf.tanh(tf.matmul(inputs, self.cx) + tf.matmul(hidden_layer, self.cm)\
                                  + tf.matmul(c, self.cc) + self.cb,
                            name="update_gate")
            state = tf.add(tf.mul(forget_gate, state), tf.mul(input_gate, update_gate),
                           name="state")
            output_gate = tf.sigmoid(tf.matmul(inputs, self.ox) + tf.matmul(hidden_layer, self.om)\
                                     + self.ob,
                                     name="output_gate") 
            output = tf.mul(output_gate, tf.tanh(state), name="output")
            
        #Note that, the "output" is equal to the hidden state
        return output, state
        
    def attention(self, h, s):
        with tf.variable_scope("attention_module", reuse=self.reuse):
            self.Ua = tf.get_variable("Ua", [int(h[0].get_shape()[1])/2, int(h[0].get_shape()[1])],
                                     tf.float32, tf.truncated_normal_initializer(stddev=0.1))
            self.Wa = tf.get_variable("Wa", [int(s.get_shape()[1]), int(s.get_shape()[1])],
                                     tf.float32, tf.truncated_normal_initializer(stddev=0.1))
            #the following bias does not showed in original paper
            self.Ba = tf.get_variable("Ba", [1, int(s.get_shape()[1])], 
                                      tf.float32,
                                      tf.constant_initializer(0.1))
            self.Va = tf.get_variable("Va", [1, int(s.get_shape()[1])], 
                                      tf.float32,
                                      tf.constant_initializer(0.1))
            attn_logits = list()
            for h_j in h:
                attn_mlp = tf.tanh(
                    tf.matmul(self.Wa, tf.transpose(s)) + tf.matmul(self.Ua, tf.transpose(h_j)))
                attn_logits.append(tf.reshape(
                        tf.matmul(self.Va, attn_mlp), shape=[1])[0])
                
            a = tf.reshape(tf.nn.softmax(attn_logits),
                           shape=[len(h)], name="attention_value")
            c = 0
            for j in range(int(a.get_shape()[0])):
                c += a[j] * h[j]
                
            return c
    
    def __call__(self):
        return self.model(self.bi_hidden, self.batch_size, self.vocabulary_size, self.num_nodes, self.max_len)

In [8]:
'''bi_h = list()
for i in range(10):
    bi_h.append(tf.Variable(tf.truncated_normal(stddev=0.5, shape=(1,10))))

gen_L = gen_LSTM(bi_h, vocabulary_size=5, batch_size=1, num_nodes=5, max_len=32, name="test_gen_lstm0")

a, b, c, d = gen_L()

sess = tf.Session()
sess.run(tf.initialize_all_variables())
feed_dict = dict()

x,y,z,w = sess.run([a,b,c, d], feed_dict=feed_dict)'''

'bi_h = list()\nfor i in range(10):\n    bi_h.append(tf.Variable(tf.truncated_normal(stddev=0.5, shape=(1,10))))\n\ngen_L = gen_LSTM(bi_h, vocabulary_size=5, batch_size=1, num_nodes=5, max_len=32, name="test_gen_lstm0")\n\na, b, c, d = gen_L()\n\nsess = tf.Session()\nsess.run(tf.initialize_all_variables())\nfeed_dict = dict()\n\nx,y,z,w = sess.run([a,b,c, d], feed_dict=feed_dict)'

In [9]:
class Discriminator():
    def __init__(self, s, conv_sizes, num_filters=64, dropout_keep_prob=0.5, 
                 name=""):
        '''
        The Discriminator, uses Convolutional Neural Network to 
        classify the input sentence. See 
        http://www.people.fas.harvard.edu/~yoonkim/data/emnlp_2014.pdf
        for detail. Those code below with some inspiration from
        [https://github.com/dennybritz/cnn-text-classification-tf/blob/master/text_cnn.py]
        
        Arguments:
        s: the sentence representation caculated by LSTM process, 
        which should a size of [batch_size, vocabulary_size, length, channel = 1]
        '''
        self.s = s
        self.sequence_length = int(s.get_shape()[1])
        self.reuse = False
        self.name = name
        self.dropout_keep_prob = dropout_keep_prob
        self.num_filters = num_filters
        self.conv_sizes = conv_sizes
        self.vocabulary_size = int(s.get_shape()[2])
        #print(self.hidden_size)
        self.pooled_outputs = list()
    
    def model(self):
        for i, conv_size in enumerate(self.conv_sizes):
            with tf.variable_scope("D_conv_maxpooling_{}"
                                  .format(conv_size), reuse=self.reuse):
                #filter_shape = [conv_size, self.hidden_size, 1, num_filters]
                W = tf.get_variable("W", [conv_size, self.vocabulary_size, 1, self.num_filters],
                                   tf.float32,
                                   tf.truncated_normal_initializer(stddev=0.1))
                print("the size of {} => {}"
                      .format(W.name, W.get_shape()))
                b = tf.get_variable("b", [self.num_filters], tf.float32,
                                   tf.constant_initializer(0.1))
                
                conv = tf.nn.conv2d(self.s, W, [1,1,1,1], 'VALID', name="conv")
                print("the size of {} => {}"
                      .format(conv.name, conv.get_shape()))
                h = tf.nn.relu(conv + b)
                
                pooled = tf.nn.max_pool(h,
                                       ksize=[1, self.sequence_length-conv_size+1,1,1],
                                       strides=[1,1,1,1],
                                       padding='VALID',
                                       name="max_pooling")
                print("the size of pooling layer {}"
                      .format(pooled.get_shape()))
            self.pooled_outputs.append(pooled)
            
        self.total_filter_num = self.num_filters * len(self.conv_sizes)
        self.h_pool = tf.concat(3, self.pooled_outputs,
                                name="{}_concat_pooling".format(self.name))
        self.h_pool_flat = tf.reshape(self.h_pool, [-1, self.total_filter_num])
        print("h_pool size => {}".format(self.h_pool_flat.get_shape()))
     
        #after convolutional layer, we can define the fully connectedmlayer
        
        with tf.variable_scope("D_fully_connected_layer", reuse=self.reuse):
            w_shape = [self.h_pool_flat.get_shape()[1], 1]
            W = tf.get_variable("W", w_shape, tf.float32,
                               tf.truncated_normal_initializer(stddev=0.1))
            b = tf.get_variable("b", [1], tf.float32,
                               tf.constant_initializer(0.1))
            logits = tf.nn.relu(tf.matmul(self.h_pool_flat, W) + b)
            print("logits size => {}".format(logits.get_shape()))
            
        self.reuse = True
        
    def __call__(self):
        return self.model()

In [10]:
class Generator:
    def __init__(self, input_sentence, max_len=32, vocabulary_size=200, 
                 num_nodes=128, batch_size=16, name=""):
        self.input_sentence = input_sentence
        self.batch_size = batch_size
        self.num_nodes = num_nodes
        self.name = name
        self.vocabulary_size = vocabulary_size
        self.reversed_sentence = self.reverse_sentence(self.input_sentence)
        
        L_forward = LSTM(self.vocabulary_size, self.num_nodes, self.batch_size, 
                         name="forward_{}".format(self.name))
        L_backward = LSTM(self.vocabulary_size, self.num_nodes, self.batch_size,
                          name="backward_{}".format(self.name))
        
        self.forward_LSTM_outputs = L_forward(self.input_sentence)
        self.backward_LSTM_outputs = L_backward(self.reversed_sentence)
        #concat the two outputs 
        self.backward_LSTM_outputs = self.reverse_sentence(self.backward_LSTM_outputs)
        self.BiLSTM_outputs = list()
        for i in range(len(self.forward_LSTM_outputs)):
            self.BiLSTM_outputs.append(
                tf.concat(
                    1,[self.forward_LSTM_outputs[i], self.backward_LSTM_outputs[i]]))
        
        #has not been debugged yet---10/14/2016
        self.gen_LSTM = gen_LSTM(self.BiLSTM_outputs, self.batch_size, 
                                 self.vocabulary_size, self.num_nodes, 
                                 max_len=10, name="GEN_{}".format(self.name))
        self.generated_sentence = self.gen_LSTM()
        
        
        
    def reverse_sentence(self, inputs):
        length = len(inputs)
        ret = list()
        #reverse
        for i in range(length):
            ret.append(inputs[length-i-1])
        return ret
    def pad_sentence(self, input_sentence):
        #add zero at the end of the sentence
        if len(input_sentence) >= MAX_FILTER_SIZE:
            return input_sentence
        else:
            for i in range(MAX_FILTER_SIZE - len(input_sentence)):
                input_sentence.append(np.zeros(WORD_SIZE))
            return input_sentence

In [11]:
word_model = gensim.models.Word2Vec.load("models/word2vec_2D.model")

test_word1 = word_model["china"]
test_word2 = word_model["uk"]
test_word3 = word_model["america"]

test_sentence = [test_word1, test_word2, test_word3]

for i, _ in enumerate(test_sentence):
    test_sentence[i] = test_sentence[i].reshape(1,2)

2016-10-15 16:32:00,116 : INFO : loading Word2Vec object from models/word2vec_2D.model
2016-10-15 16:32:00,326 : INFO : setting ignored attribute syn0norm to None
2016-10-15 16:32:00,331 : INFO : setting ignored attribute cum_table to None


In [12]:
G = Generator(test_sentence, max_len=4, vocabulary_size=2, num_nodes=5, batch_size=1, name="G3")

input word at step 0:Tensor("Cast:0", shape=(1, 2), dtype=float32)
predicted word:Tensor("ArgMax:0", shape=(1,), dtype=int64)
at step 1 predicted word embedding:Tensor("Cast_2:0", shape=(1, 2), dtype=float32)
predicted word:Tensor("ArgMax_1:0", shape=(1,), dtype=int64)
at step 2 predicted word embedding:Tensor("Cast_4:0", shape=(1, 2), dtype=float32)
predicted word:Tensor("ArgMax_2:0", shape=(1,), dtype=int64)
at step 3 predicted word embedding:Tensor("Cast_6:0", shape=(1, 2), dtype=float32)
predicted word:Tensor("ArgMax_3:0", shape=(1,), dtype=int64)
at step 4 predicted word embedding:Tensor("Cast_8:0", shape=(1, 2), dtype=float32)
predicted word:Tensor("ArgMax_4:0", shape=(1,), dtype=int64)
at step 5 predicted word embedding:Tensor("Cast_10:0", shape=(1, 2), dtype=float32)
predicted word:Tensor("ArgMax_5:0", shape=(1,), dtype=int64)
at step 6 predicted word embedding:Tensor("Cast_12:0", shape=(1, 2), dtype=float32)
predicted word:Tensor("ArgMax_6:0", shape=(1,), dtype=int64)
at step 

In [13]:
D = Discriminator(s_d, [3], 64, 0.5, False,"ddd1d")

NameError: name 's_d' is not defined

In [ ]:
#L = LSTM(batch_size=1, num_unrollings=2, name="TEST")
A = attention(h_j, s_j, "143")
word_model = gensim.models.Word2Vec.load("models/word2vec.model")

test_word1 = word_model["china"]
test_word2 = word_model["uk"]
test_word3 = word_model["america"]

test_sentence = [test_word1, test_word2, test_word3]

with tf.Session() as sess:
    sess.run(tf.initialize_all_variables())
    feed_dict = dict()
    #for i in range(3):
        #feed_dict[L.train_data[i]] = np.reshape(test_sentence[i],(1,200))
    outc, outa = sess.run([A.c, A.a], feed_dict=feed_dict)